In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score, jaccard_score

In [2]:
dir = 'E:/UMINTv2/Data/preprocessedData/MALT10k/'
dir1 = 'E:/UMINTv2/'

In [3]:
#Importing raw data
rna = pd.read_csv(dir+'MALT10k_rna_scaled.csv',header = 0, index_col= 0)
adt = pd.read_csv(dir+'MALT10k_adt_scaled.csv',header = 0, index_col= 0)

x1 = rna.T
x2 = adt.T

z = pd.concat([x1, x2], axis=1)

In [4]:
labels = pd.read_csv(dir1+'Labels/MALT10k/MALT10k_groundTruth.csv',header = 0 , index_col= 0)
seuratLabels = pd.read_csv(dir1+'Labels/MALT10k/MALT10k_labels_Seurat.csv',header = 0 , index_col= 0)
mofa2embedding = pd.read_csv(dir1+'Data/Embedding/MALT10k_MOFA2_embedding.csv',header = 0 , index_col= 0)
totalVILabels = pd.read_csv(dir1+'Labels/MALT10k/MALT10k_labels_totalVI.csv',header = 0 , index_col= 0)

Encoding labels

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = pd.DataFrame(le.fit_transform(np.ravel(labels)))

In [6]:
num_clusters = len(np.unique(y))

In [7]:
result={}

**Seurat**

In [8]:
l=[]
l.append(adjusted_rand_score(y[0], seuratLabels['seurat_clusters']))
l.append(fowlkes_mallows_score(y[0], seuratLabels['seurat_clusters']))
result['Seurat']=l

**MOFA+**

In [9]:
labels_mofa2 = KMeans(n_clusters= num_clusters).fit_predict(mofa2embedding)

In [10]:
l=[]
l.append(adjusted_rand_score(y[0], labels_mofa2))
l.append(fowlkes_mallows_score(y[0], labels_mofa2))
result['MOFA2']=l

**TotalVI**

In [11]:
totalVILabels = totalVILabels['leiden_totalVI']

In [12]:
l=[]
l.append(adjusted_rand_score(y[0], totalVILabels))
l.append(fowlkes_mallows_score(y[0], totalVILabels))
result['TotalVI']=l

Exporting results

In [13]:
res= pd.DataFrame(result)
res=res.T
res.columns = ['ARI','FMI']
res

,ARI,FMI
Seurat,0.448348,0.526503
MOFA2,0.273347,0.527707
TotalVI,0.458741,0.533776


In [14]:
res.to_csv(dir1+'Results/MALT10k/MALT10k_comparativeAnalysis_kmeans.csv')